# Tópicos de Industria I
## Práctica 3 - Métricas de Caminados aleatorios

<img style="margin: 10px" src="./public/banner.png" alt="Assignment Banner" height="180" width="980" />

**E-mail:** roberto.carrillo7958@alumnos.udg.mx

**Ciclo:** 2023-A

**Fecha:** 17/02/2023

## MODULES

In [2]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

## CLASSES

In [3]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

In [129]:
# HELPER FUNCTIONS

def plot_2d_scatter(x_data, y_data, traj_name, fig):
    fig.add_trace(go.Scatter(
        x = x_data,
        y = y_data,
        marker = dict( size = 2 ),
        line = dict( width = 2 ),
        name = f"{traj_name}",
        showlegend = True
    ))

def plot_2d_histogram(data_vector, hist_name, fig, bins = np.pi/300):
    fig.add_trace(go.Histogram(
        x = data_vector,
        xbins = dict(size=bins),
        histnorm = 'percent',
        name = f'{hist_name}',
        opacity = 0.80,
        showlegend=True))


## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Cargar trayectorias en **Pandas** Data Frame
* Calcular métrica utilizando exclusivamente funciones de NumPy
* Guardar métricas en **Pandas** Data Frame
* Visualizar con **plotly**

### Implementación

 1. Se tienen que cargar las trayectorias, donde usamos un archivo csv para transformarlo a Dataframe
 2. Se define una función que permite calcular la distancia entre dos puntos:
      1. Calculamos el cuadrado de la diferencia del eje correspondiente
      2. Retornamos la raíz de la suma de dichos cuadrados
 3. Creamos una función que itera sobre cada posición de la trayectoria y calcula la distancia de cada punto, retornando la suma acumulativa de toda la trayectoria
 4. Se genera una función para graficar, y se renderiza

In [4]:
# Load existing trajectories
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [5]:
# Creación de función para cálculo de distancia entre dos vectores
def calc_dist_2vectors(point1, point2):

    x_diff = np.square(point2.x_pos - point1.x_pos)
    y_diff = np.square(point2.y_pos - point1.y_pos)
    vectors_distance = np.sqrt(x_diff + y_diff)
    
    return vectors_distance

# Creación de función para calcular la longitud de la trayectoria
# Haciendo uso de la función creada anteriormente, vamos iterando sobre cada posición
# Y acumulando la suma de cada distancia entre el vector actual y el anterior
def create_pl_bm_2d(trajectory_df):
    pl_distance_bm_df = np.array([
        calc_dist_2vectors(
            trajectory_df.iloc[i - 1],
            trajectory_df.iloc[i]
        ) for i in range(1, trajectory_df.shape[0])
    ])
    return np.cumsum(pl_distance_bm_df)


In [14]:
# Define trajectories array
trajectories = [
    { "traj_name": "BM_2d_df_3", "traj_data": BM_2d_df_3 },
    { "traj_name": "BM_2d_df_6", "traj_data": BM_2d_df_6 },
    { "traj_name": "CRW_2d_df_9", "traj_data": CRW_2d_df_9 }
]
trajectories_df = pd.DataFrame(columns=['traj_name', 'traj_data', 'path_length', 'path_length_data'])

for traj in trajectories:
    path_length = create_pl_bm_2d(traj['traj_data'])
    temp_df = pd.DataFrame([{
        "traj_name": traj['traj_name'],
        "traj_data": traj['traj_data'],
        "path_length": path_length[-1],
        "path_length_data": path_length,
    }])
    trajectories_df = pd.concat([trajectories_df, temp_df], axis=0, ignore_index=True)



In [115]:
# Plotting
fig_path_length = go.Figure()

for index, row in trajectories_df.iterrows():
    plot_2d_scatter(
        x_data = np.arange(len(row.path_length_data)),
        y_data = row.path_length_data, 
        traj_name = row.traj_name, 
        fig = fig_path_length)

fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW) (8 pts)

* Cargar trayectorias en **Pandas** Data Frame
* Guardar metricas en **Pandas** Data Frame
* Visualizar con **plotly**


El Mean Squared Displacement es una métrica que entrega más información acerca de la trayectoria de una partícula. Mientras que el largo del recorrido se encarga de medir toda la distancia que se recorrió a lo largo de la trayectoria.



In [15]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
        y = BM_2d_df_6.y_pos,
        z = BM_2d_df_6.index,
        marker = dict(size=2),
        line = dict(color='blue', width=2),
        mode = 'lines',
        name = 'BM 2d',
        showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
        y = CRW_2d_df_9.y_pos,
        z = CRW_2d_df_9.index,
        marker = dict(size=2),
        line = dict(color='red', width=2),
        mode = 'lines',
        name = 'CRW 2d',
        showlegend = True))

fig_3d.show()

In [109]:
# Function to calculate the displacement vector across n_step 
def calc_displacement_vec(trajectory, n_step):
    return np.array([
            np.square(distance.euclidean(trajectory.iloc[j], trajectory.iloc[j - n_step]))
        for j in range(n_step, len(trajectory))])

# Function to create the Mean Squared Displacement vector, looping through every t point in order to get the mean
# This result is saved into a numpy array creating the output vector
def create_msd_vec(trajectory):
    MSD_Out = np.empty(0)
    traj_length = trajectory.shape[0]
    for tau in range(1, traj_length):
        n_step = tau
        msd = np.mean(calc_displacement_vec(trajectory, n_step))
        MSD_Out = np.append(MSD_Out,msd)
    return MSD_Out

MSD_BM = create_msd_vec(BM_2d_df_6)
MSD_CRW = create_msd_vec(CRW_2d_df_9)
msd_bm_df = pd.DataFrame(MSD_BM, columns=["MSD_BM"])
msd_crw_df = pd.DataFrame(MSD_CRW, columns=["MSD_CRW"])

met_df_2 = msd_bm_df.join(msd_crw_df)


In [116]:
# Plot MSD Vectors
fig_path_length = go.Figure()

plot_2d_scatter(np.arange(len(MSD_BM)),MSD_BM, "MSD_BM", fig_path_length)
plot_2d_scatter(np.arange(len(MSD_CRW)), MSD_CRW, "MSD_CRW", fig_path_length)

fig_path_length.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (9 pts)

* Generar CRWs con dos exponentes diferentes
* Guardar trayectorias en Pandas Data Frame
* Obtener Turning-angle distribution
* Guardar metricas en **Pandas** Data Frame
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [56]:
# Load existing trajectories
# CRW speed = 6, 
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_6.csv')

# Load existing trajectories
# CRW speed = 6, 
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [83]:
def calc_2vec_angle(p1, p2, p3):
    # Calc ab, bc and it's norm values
    ab = p2 - p1
    bc = p3 - p2
    norm_ab = np.linalg.norm(ab)
    norm_bc = np.linalg.norm(bc)

    # Calc dot and cross product of both segments
    dot_p = np.dot(ab, bc)
    cross_p = np.cross(ab, bc)
    # Check direction of vector
    direction = np.sign(cross_p)
    if direction == 0:
        direction = 1
    # Get cos angle, add eps to avoid NaN    
    cos_angle = dot_p/((norm_ab * norm_bc) + np.finfo(float).eps)
    turn_angle = np.arccos(cos_angle) * direction
    return turn_angle

def calc_turn_angle_dist(trajectory):
    # Generate out array
    out_array = np.empty(0)
    # Iterate over trajectory, get turn angle and append to out_array
    for index in range(1, trajectory.shape[0] - 1):
        ## start - Add your code here
        p1 = trajectory.iloc[index-1]
        p2 = trajectory.iloc[index]
        p3 = trajectory.iloc[index+1]
        calculated_angle = calc_2vec_angle(p1,p2,p3)
        out_array= np.append(out_array, calculated_angle)
    return out_array

# Create aux arrays to store angle values
aux_ta_CRW_2d_df_6 = calc_turn_angle_dist(CRW_2d_df_6)
aux_ta_CRW_2d_df_9 = calc_turn_angle_dist(CRW_2d_df_9)

# Save to pandas DF
met_crw_df= pd.DataFrame()
temp_crw_df= pd.DataFrame()
met_crw_df['TA_CRW_6'] = aux_ta_CRW_2d_df_6
temp_crw_df['TA_CRW_9'] = aux_ta_CRW_2d_df_9
met_crw_6_9_df=pd.merge(met_crw_df,temp_crw_df, left_index=True, right_index=True,how='inner')


# # Write to csv
# met_crw_6_9_df.to_csv("./met_crw_6_9.csv") 


In [124]:
# PLot histogram
fig_met_df_3 = go.Figure()

plot_2d_histogram(met_crw_6_9_df.TA_CRW_6, "observed_0.6", fig_met_df_3)
plot_2d_histogram(met_crw_6_9_df.TA_CRW_9, "observed_0.9", fig_met_df_3)

# Cauchy Dist aux Vector params
resolution = 300
aux_domain = np.linspace(0, 2*np.pi, resolution)

for CRW_exponent in [0.6, 0.9]:
  wrapcauchy_pdf = np.array([wrapcauchy.pdf(i, CRW_exponent) for i in aux_domain])

  # Plot Aux Vector
  aux_plot = np.linspace(-np.pi, np.pi, resolution)

  plot_wrapcauchy_pdf = np.concatenate((wrapcauchy_pdf[int (resolution/2):resolution], wrapcauchy_pdf[0:int(resolution/2)]),  axis=0)
  plot_2d_scatter(aux_plot, plot_wrapcauchy_pdf, f'origin_{CRW_exponent}', fig_met_df_3)

fig_met_df_3.update_layout(
    title_text='Turning angle distribution for CRW & BM - (Origin vs Observed)'
)
fig_met_df_3.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (9 pts)

* Generar Levy Flight
* Guardar trayectorias en **Pandas** Data Frame
* Guardar metricas en **numpy** array
* Obtener Step-length distribution
* Comparar en gráfica distribución origen vs distribución observada
* Visualizar con **plotly**

In [119]:
# Load existing trajectories
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Load existing trajectories
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [ ]:

# aux to store turning angles
aux_ta_Levy_2d_df_1 = calc_turn_angle_dist(Levy_2d_df_1)
# aux to store step-lengths
aux_sl_Levy_2d_df_1 = np.empty(shape=(0))

# aux to store turning angles
aux_ta_Levy_2d_df_7  = calc_turn_angle_dist(Levy_2d_df_1)
# aux to store step-lengths
aux_sl_Levy_2d_df_7 = np.empty(shape=(0))

calc_abs = lambda x: np.abs(x)

In [188]:
# PLOT Distributions

# Distribution Parameters
fig_met_df_4 = go.Figure()


loc = 3.0
beta = 1.0
resolution = 400
aux_domain=np.linspace(0,20, resolution)

for Levy_exp in [0.7, 1.0]:
    Levy_pdf=np.array([levy_stable.pdf(i, Levy_exp, beta, loc) for i in aux_domain])
    plot_2d_scatter(aux_domain, Levy_pdf, f'Levy_{Levy_exp}', fig_met_df_4)
    # levy_rvs = levy_stable.rvs(Levy_exp, beta, loc=loc, size=4000)
    # plot_2d_histogram(levy_rvs, f'Observed_{Levy_exp}', fig_met_df_4, 40)

fig_met_df_4.show()